# Spec Kit Operations Guide (Copilot in Codespaces)

Deep exploration of Spec Kit and safe bootstrap of a .NET Aspire product search template with Copilot Agent Mode.

In [1]:
# Section 1: Detect OS, workspace roots, and safe execution flags
import os, sys, platform, json, tempfile, pathlib
from typing import Dict

SAFE_MODE = True  # hard-default: never execute destructive actions automatically
DRY_RUN = True    # default: print commands instead of running

OS_NAME = platform.system().lower()  # 'linux', 'darwin', or 'windows'
ROOT = pathlib.Path('/')
WORKSPACE_ROOT = pathlib.Path(os.environ.get('WORKSPACE_ROOT', '/workspaces/ActualGameSearch_V3')).resolve()
SANDBOX_DIR = pathlib.Path(tempfile.mkdtemp(prefix='spec_kit_sandbox_'))
ARTIFACTS_DIR = WORKSPACE_ROOT / 'AI-Agent-Workspace' / 'Artifacts'
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)

config: Dict[str, str] = {
    'OS_NAME': OS_NAME,
    'WORKSPACE_ROOT': str(WORKSPACE_ROOT),
    'SANDBOX_DIR': str(SANDBOX_DIR),
    'ARTIFACTS_DIR': str(ARTIFACTS_DIR),
    'SAFE_MODE': str(SAFE_MODE),
    'DRY_RUN': str(DRY_RUN),
}
print(json.dumps(config, indent=2))

{
  "OS_NAME": "linux",
  "WORKSPACE_ROOT": "/workspaces/ActualGameSearch_V3",
  "SANDBOX_DIR": "/tmp/spec_kit_sandbox_c3rzraok",
  "ARTIFACTS_DIR": "/workspaces/ActualGameSearch_V3/AI-Agent-Workspace/Artifacts",
  "SAFE_MODE": "True",
  "DRY_RUN": "True"
}


In [2]:
# Section 2: Verify prerequisites: Python 3.11+, uv, git, dotnet, Node, PowerShell/bash
import subprocess, shutil

def check_cmd(cmd, args=["--version"], capture_output=True):
    try:
        proc = subprocess.run([cmd, *args], capture_output=capture_output, text=True, check=True)
        return True, proc.stdout.strip() or proc.stderr.strip()
    except FileNotFoundError:
        return False, f"{cmd} not found"
    except subprocess.CalledProcessError as e:
        return False, e.stdout or e.stderr

results = {}
# Python version
py_ver = sys.version.split()[0]
results['python'] = (py_ver >= '3.11', f"Python {py_ver}")
# uv
ok, out = check_cmd('uv', ['--version'])
results['uv'] = (ok, out)
# git
ok, out = check_cmd('git', ['--version'])
results['git'] = (ok, out)
# dotnet
ok, out = check_cmd('dotnet', ['--info'])
results['dotnet'] = (ok, out[:500])
# node
ok, out = check_cmd('node', ['--version'])
results['node'] = (ok, out)
# powershell or bash
if OS_NAME.startswith('windows'):
    ok, out = check_cmd('pwsh', ['--version'])
    results['shell'] = (ok, out if ok else 'PowerShell (pwsh) not found')
else:
    ok, out = check_cmd('bash', ['--version'])
    results['shell'] = (ok, out.split('\n')[0] if ok else 'bash not found')

print(json.dumps({k: {"ok": v[0], "detail": v[1]} for k,v in results.items()}, indent=2))

assert results['python'][0], "Python 3.11+ required"
assert results['uv'][0], "uv is required"
assert results['git'][0], "git is required"
assert results['dotnet'][0], "dotnet SDK required (for Aspire)"

{
  "python": {
    "ok": true,
    "detail": "Python 3.12.3"
  },
  "uv": {
    "ok": true,
    "detail": "uv 0.8.19"
  },
  "git": {
    "ok": true,
    "detail": "git version 2.50.1"
  },
  "dotnet": {
    "ok": true,
    "detail": ".NET SDK:\n Version:           8.0.412\n Commit:            819e1a9566\n Workload version:  8.0.400-manifests.9cf71931\n MSBuild version:   17.11.31+933b72e36\n\nRuntime Environment:\n OS Name:     ubuntu\n OS Version:  24.04\n OS Platform: Linux\n RID:         linux-x64\n Base Path:   /usr/share/dotnet/sdk/8.0.412/\n\n.NET workloads installed:\nConfigured to use loose manifests when installing new manifests.\nThere are no installed workloads to display.\n\nHost:\n  Version:      8.0.18\n  Architecture: x64\n  Comm"
  },
  "node": {
    "ok": true,
    "detail": "v22.17.0"
  },
  "shell": {
    "ok": true,
    "detail": "GNU bash, version 5.2.21(1)-release (x86_64-pc-linux-gnu)"
  }
}


In [ ]:
# Section 3: Install/upgrade uv and prepare Spec Kit CLI invocation
import sys

def ensure_uv():
    ok, _ = check_cmd('uv', ['--version'])
    if ok:
        print('uv present; skipping bootstrap')
        return
    if DRY_RUN:
        print('[DRY_RUN] Would install uv via official installer')
        return
    # Safe installer (non-destructive); prefer curl script from Astral docs when permitted
    try:
        subprocess.run(['curl','-LsSf','https://astral.sh/uv/install.sh','-o','install.sh'], check=True)
        subprocess.run(['sh','install.sh'], check=True)
    finally:
        if os.path.exists('install.sh'):
            os.remove('install.sh')

ensure_uv()

SPECIFY = ['uvx','--from','git+https://github.com/github/spec-kit.git','specify']
print('CLI prepared:', ' '.join(SPECIFY))

In [ ]:
# Section 4: Run "specify check" with debug logs (non-destructive)
import shlex

def run_specify_check(ignore_agent_tools=False):
    cmd = [*SPECIFY, 'check']
    if ignore_agent_tools:
        cmd.append('--ignore-agent-tools')
    print('Running:', shlex.join(cmd))
    if DRY_RUN:
        print('[DRY_RUN] Skipping execution')
        return {"dry_run": True}
    proc = subprocess.run(cmd, text=True, capture_output=True)
    print(proc.stdout)
    if proc.returncode != 0:
        print(proc.stderr)
    return {
        'code': proc.returncode,
        'stdout': proc.stdout,
        'stderr': proc.stderr,
    }

check_result = run_specify_check(ignore_agent_tools=False)

In [ ]:
# Section 5: Bootstrap a sandbox Spec Kit project in a temp dir (Copilot, PS/sh)
from pathlib import Path

def spec_init_sandbox(project_name='product-search-template'):
    target = Path(SANDBOX_DIR) / project_name
    target.mkdir(parents=True, exist_ok=True)
    script = 'ps' if OS_NAME.startswith('windows') else 'sh'
    cmd = [*SPECIFY, 'init', project_name, '--ai', 'copilot', '--script', script, '--no-git', '--debug']
    print('Init cmd:', ' '.join(cmd))
    if DRY_RUN:
        print('[DRY_RUN] Skipping init execution in', target)
        return str(target)
    proc = subprocess.run(cmd, cwd=SANDBOX_DIR, text=True)
    if proc.returncode != 0:
        raise RuntimeError('specify init failed')
    return str(target)

sandbox_path = spec_init_sandbox()

In [ ]:
# Section 6: Inspect scaffolded project tree and important templates
import os

def filtered_tree(root: str, max_depth=3):
    root_path = Path(root)
    lines = []
    for dirpath, dirnames, filenames in os.walk(root_path):
        depth = Path(dirpath).relative_to(root_path).parts
        if len(depth) > max_depth:
            continue
        rel = str(Path(dirpath).relative_to(root_path)) or '.'
        lines.append(f"[DIR] {rel}")
        for f in filenames:
            lines.append(f"      - {rel}/{f}")
    return "\n".join(lines)

print(filtered_tree(sandbox_path))

# Key files to verify
key_files = [
    'templates/spec-template.md',
    'templates/plan-template.md',
    'templates/tasks-template.md',
    'scripts/bash/create-new-feature.sh',
]
for k in key_files:
    p = Path(sandbox_path) / k
    print(k, 'exists?' , p.exists())

In [ ]:
# Section 7: Generate memory/constitution.md (no terminal during /constitution)
from textwrap import dedent

def write_text(path: Path, content: str):
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(content, encoding='utf-8')

constitution = dedent('''
# Project Constitution: .NET Aspire Product Search Template

Principles
- Quality: CI-enforced build, lint, tests; PR review required.
- Testing: Unit tests (xUnit), minimal integration tests for APIs, perf smoke for search endpoints.
- UX: Fast, predictable, accessible; explainability for search results.
- Performance: P95 under 300ms for top-10 results with warm cache; memory and CPU budgets documented.
- Security: No secrets in repo; dev uses .env with VS Code settings; keyvault/managed identity in cloud.
- AI Agent Guardrails: Slash commands only; never run destructive terminal ops for /constitution, /specify, /plan, /tasks, /implement.

Governance
- Architecture decisions recorded (ADR-YYYYMMDD.md).
- Dependency updates pinned with changelogs; breaking changes gated by tests.
- Observability: structured logs, health endpoints, minimal traces.
''')

constitution_path = Path(sandbox_path) / '.specify' / 'memory' / 'constitution.md'
write_text(constitution_path, constitution)
print('Wrote constitution to', constitution_path)

In [ ]:
# Section 8: Create specs/001-product-search/spec.md from template with AGS-aligned requirements
feature_dir = Path(sandbox_path) / 'specs' / '001-product-search'
feature_dir.mkdir(parents=True, exist_ok=True)

spec_md = dedent('''
# Feature: Product/Game Search (Hybrid Semantic + Full-Text)

## Problem & Goal
Users struggle to discover games relevant to their tastes. Provide high-relevance search and relatedness navigation with explainability.

## User Stories
- As a player, I can search by free text and see highly relevant games ranked by a hybrid score.
- As a player, I can see why a result matched (top reviews/snippets, tags, and proximity).
- As a player, I can pivot to "similar games" from any result to explore relatedness.
- As an admin, I can refresh embeddings and indexes without downtime.

## Non-Goals
- Payments, accounts, or personalization (initial phase).

## Acceptance Criteria
- Query returns <= 300ms P95 for top-10 results locally.
- Results list includes title, image, short description, top-3 evidence snippets (review excerpts), and similarity score.
- Similar games endpoint returns 10 related games based on vector neighbors.
- Errors are surfaced with actionable messages; no stack traces to users.

## Review & Acceptance Checklist
- [ ] Functional flows verified for search and relatedness
- [ ] Performance targets validated (P95) on seed dataset
- [ ] Accessibility checks on labels/contrast
- [ ] Logging for queries and errors (PII-free)
''')

spec_path = feature_dir / 'spec.md'
write_text(spec_path, spec_md)
print('Wrote spec to', spec_path)

In [ ]:
# Section 9: Create plan.md targeting .NET Aspire, Postgres/SQLite, Blazor, REST
plan_md = dedent('''
# Implementation Plan: .NET Aspire Product/Game Search

## Tech Stack
- .NET 10 + .NET Aspire for orchestration
- Postgres (prod) / SQLite (local dev) for metadata
- Cosmos DB or Postgres pgvector (optional) for vectors in prod; SQLite w/ FTS5 + local ANN (dev)
- Blazor Server for UI; REST APIs for Search and Similar endpoints
- Ollama (Gemma) for embeddings (local dev)

## Services
- Gateway: reverse proxy + auth stub
- API.Search: hybrid search (vector + lexical), explainability
- Workers.ETL: fetch Steam metadata/reviews; embed text; write to DB
- Shared: contracts/models

## Versions
- .NET SDK: 10.x (confirm with `dotnet --info`)
- Aspire: latest stable compatible with SDK
- Postgres: 16.x; pgvector 0.6+

## Infra
- Local dev uses docker-compose: Postgres, Ollama
- Observability: structured logs (Serilog), health checks, minimal tracing

## Rationale
.NET Aspire simplifies local orchestration; Postgres provides low-friction persistence; Ollama enables no-cost local embeddings.
''')

plan_path = feature_dir / 'plan.md'
write_text(plan_path, plan_md)
print('Wrote plan to', plan_path)

In [ ]:
# Section 10: Derive tasks.md from plan with TDD and dependencies
tasks_md = dedent('''
# Tasks

## Order & Dependencies
1. Scaffold Aspire solution [id:scaffold]
2. Add API.Search project [id:api] depends_on: scaffold
3. Add Workers.ETL project [id:etl] depends_on: scaffold
4. Add Shared contracts [id:shared] depends_on: scaffold
5. Docker compose for Postgres + Ollama [id:compose] depends_on: scaffold
6. Search API endpoints + tests [id:api-tests] depends_on: api, shared
7. ETL pipelines + tests [id:etl-tests] depends_on: etl, shared
8. Blazor UI skeleton + smoke tests [id:ui] depends_on: api

## TDD Steps (examples)
- For Search API: write tests for ranking composition (proximity + resonance) before implementation.
- For ETL: tests for weighted game vector from review embeddings.
- For UI: component tests for results list and evidence rendering.
''')

tasks_path = feature_dir / 'tasks.md'
write_text(tasks_path, tasks_md)
print('Wrote tasks to', tasks_path)

In [ ]:
# Section 11: Validate project readiness
from collections import deque

required = [constitution_path, spec_path, plan_path, tasks_path]
missing = [str(p) for p in required if not Path(p).exists()]
print('Missing:', missing)

# Simple DAG check from tasks file
import re
edges = []
ids = set()
for line in tasks_md.splitlines():
    m_id = re.search(r"\[id:([a-zA-Z0-9_-]+)\]", line)
    if m_id:
        ids.add(m_id.group(1))
    m_dep = re.search(r"depends_on:\s*([^\n]+)", line)
    if m_dep and m_id:
        deps = [d.strip() for d in m_dep.group(1).split(',')]
        for d in deps:
            edges.append((d, m_id.group(1)))

# Kahn's algorithm
indeg = {i:0 for i in ids}
for u,v in edges:
    if v in indeg:
        indeg[v]+=1
q = deque([i for i,d in indeg.items() if d==0])
order = []
while q:
    u=q.popleft(); order.append(u)
    for a,b in list(edges):
        if a==u:
            indeg[b]-=1
            edges.remove((a,b))
            if indeg[b]==0:
                q.append(b)

acyclic = len(edges)==0
print(json.dumps({
  'files_present': len(missing)==0,
  'dag_acyclic': acyclic,
  'topo_order_prefix': order[:5]
}, indent=2))

In [ ]:
# Section 12: Emit Copilot slash-command payloads for manual paste
constitution_prompt = "/constitution Create principles focused on code quality (CI tests), UX consistency and accessibility, performance (P95<300ms for search), security (no secrets), and AI-agent guardrails (no terminal ops for slash commands)."
specify_prompt = "/specify Build a hybrid semantic + full-text product/game search with explainability, similar-games navigation, and seed dataset support. Return top-10 with evidence snippets and similarity."
plan_prompt = "/plan Use .NET Aspire (.NET 10), Blazor Server, Postgres (local SQLite), and Ollama for embeddings. Provide Search API, Similar API, and ETL workers. Include versions and docker-compose."
tasks_prompt = "/tasks"
implement_prompt = "/implement"

print('\n'.join([
    'Paste into Copilot Agent Mode:',
    constitution_prompt,
    specify_prompt,
    plan_prompt,
    tasks_prompt,
    implement_prompt
]))

In [ ]:
# Section 13: Optional: Execute repo wiring scripts with dry-run
ps_script = Path(sandbox_path) / 'scripts' / 'powershell' / 'setup-plan.ps1'
sh_script = Path(sandbox_path) / 'scripts' / 'bash' / 'setup-plan.sh'

print('PowerShell script exists?', ps_script.exists())
print('Bash script exists?', sh_script.exists())

if not DRY_RUN:
    if OS_NAME.startswith('windows') and ps_script.exists():
        subprocess.run(['pwsh','-File', str(ps_script)], check=False)
    elif sh_script.exists():
        subprocess.run(['bash', str(sh_script)], check=False)
else:
    print('[DRY_RUN] Skipping script execution')

In [ ]:
# Section 14: Export artifacts (zip) and manifest
import zipfile

zip_path = Path(ARTIFACTS_DIR) / 'spec_kit_sandbox.zip'
manifest = {
    'sandbox_path': sandbox_path,
    'constitution': str(constitution_path),
    'spec': str(spec_path),
    'plan': str(plan_path),
    'tasks': str(tasks_path),
}

if not DRY_RUN:
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as z:
        for dirpath, _, filenames in os.walk(sandbox_path):
            for f in filenames:
                p = Path(dirpath) / f
                z.write(p, arcname=str(Path(sandbox_path).name / p.relative_to(sandbox_path)))
else:
    print('[DRY_RUN] Skipping zip creation')

(Path(ARTIFACTS_DIR) / 'manifest.json').write_text(json.dumps(manifest, indent=2), encoding='utf-8')
print('Artifacts at:', ARTIFACTS_DIR)
print('Manifest:', json.dumps(manifest, indent=2))

In [ ]:
# Section 15: Minimal tests for validators and DAG

def test_files_present():
    assert Path(constitution_path).exists()
    assert Path(spec_path).exists()
    assert Path(plan_path).exists()
    assert Path(tasks_path).exists()


def test_dag_acyclic():
    # reuse 'acyclic' from earlier cell by recomputing quickly
    import re
    edges = []
    ids = set()
    for line in tasks_md.splitlines():
        m_id = re.search(r"\[id:([a-zA-Z0-9_-]+)\]", line)
        if m_id:
            ids.add(m_id.group(1))
        m_dep = re.search(r"depends_on:\s*([^\n]+)", line)
        if m_dep and m_id:
            deps = [d.strip() for d in m_dep.group(1).split(',')]
            for d in deps:
                edges.append((d, m_id.group(1)))
    indeg = {i:0 for i in ids}
    for u,v in edges:
        if v in indeg:
            indeg[v]+=1
    from collections import deque
    q = deque([i for i,d in indeg.items() if d==0])
    seen=0
    while q:
        u=q.popleft(); seen+=1
        for a,b in list(edges):
            if a==u:
                indeg[b]-=1
                edges.remove((a,b))
                if indeg[b]==0:
                    q.append(b)
    assert len(edges)==0

# Run tests
try:
    test_files_present(); test_dag_acyclic()
    print('All tests passed')
except AssertionError as e:
    print('Tests failed:', e)